Source:-https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras

In [ ]:
## Step 1:
Train the model 

In [33]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Train a model for MNIST without pruning


In [3]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [4]:
# Normalize the input image so that each pixel value is between 0 and 1
train_images = train_images/ 255.0
test_images = test_images/255.0

# Define the model

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10),
  keras.layers.Dense(10)  
])

2023-10-12 23:40:40.589026: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [6]:
model.fit(train_images,
         train_labels,
         epochs=10,
         validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.3152 - accuracy: 0.9093 - val_loss: 0.1201 - val_accuracy: 0.9692
Epoch 2/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1158 - accuracy: 0.9661 - val_loss: 0.0786 - val_accuracy: 0.9805
Epoch 3/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0820 - accuracy: 0.9757 - val_loss: 0.0682 - val_accuracy: 0.9802
Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0674 - accuracy: 0.9801 - val_loss: 0.0691 - val_accuracy: 0.9818
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0575 - accuracy: 0.9831 - val_loss: 0.0630 - val_accuracy: 0.9820
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0492 - accuracy: 0.9849 - val_loss: 0.0580 - val_accuracy: 0.9845
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0438 - accuracy: 0.9867 - val_loss: 0.0568 - val_ac

Evaluate baseline test accuracy and save the model for later usage.



In [10]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', '/afs/cern.ch/user/s/sraj/sraj/Work_/CUA_20--/MLPF/Task_Q_P/')

Baseline test accuracy: 0.9836999773979187
Saved baseline model to: /afs/cern.ch/user/s/sraj/sraj/Work_/CUA_20--/MLPF/Task_Q_P/


### Fine-tune pre-trained model with pruning
#### Define the model 
You will apply pruning to the whole model and see this in the model summary.

In this example, you start the model with 50% sparsity (50% zeros in weights) and end with 80% sparsity.

In the [comprehensive guide](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md), you can see how to prune some layers for model accuracy improvements.

In [12]:
import tensorflow_model_optimization as tfmot


prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 


num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs


# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

/cvmfs/sft.cern.ch/lcg/views/LCG_103cuda/x86_64-centos9-gcc11-opt/lib/python3.9/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:233: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.pruning_step = self.add_variable(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshape  (None, 28, 28, 1)        1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_conv2d   (None, 26, 26, 12)       230       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_max_poo  (None, 13, 13, 12)       1         
 ling2d (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_flatten  (None, 2028)             1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense (  (None, 10)               4

/cvmfs/sft.cern.ch/lcg/views/LCG_103cuda/x86_64-centos9-gcc11-opt/lib/python3.9/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:212: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  mask = self.add_variable(
/cvmfs/sft.cern.ch/lcg/views/LCG_103cuda/x86_64-centos9-gcc11-opt/lib/python3.9/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:219: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  threshold = self.add_variable(


## Train and evaluate the model against baseline
#### Fine tune with pruning for two epochs.

[tfmot.sparsity.keras.UpdatePruningStep](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/UpdatePruningStep) is required during training, and [tfmot.sparsity.keras.PruningSummaries](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/PruningSummaries) provides logs for tracking progress and debugging.

In [13]:
logdir = tempfile.mkdtemp()

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)
]

model_for_pruning.fit(train_images, train_labels, 
                     batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                     callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 7s 9ms/step - loss: 0.1620 - accuracy: 0.9520 - val_loss: 0.2014 - val_accuracy: 0.9428
Epoch 2/2
422/422 [==============================] - 3s 8ms/step - loss: 0.2008 - accuracy: 0.9414 - val_loss: 0.1448 - val_accuracy: 0.9607


For this example, there is minimal loss in test accuracy after pruning, compared to the baseline.



In [14]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9836999773979187
Pruned test accuracy: 0.9516000151634216


The logs show the progression of sparsity on a per-layer basis.



In [15]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

### Create 3x smaller models from pruning

Both [tfmot.sparsity.keras.strip_pruning](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/strip_pruning) and applying a standard compression algorithm (e.g. via gzip) are necessary to see the compression benefits of pruning.

* strip_pruning is necessary since it removes every tf.Variable that pruning only needs during training, which would otherwise add to model size during inference
* Applying a standard compression algorithm is necessary since the serialized weight matrices are the same size as they were before pruning. However, pruning makes most of the weights zeros, which is added redundancy that algorithms can utilize to further compress the model.

First, create a compressible model for TensorFlow.



In [16]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/sraj/tmp4g0h8tr4.h5


Then, create a compressible model for TFLite.



In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

2023-10-13 00:17:10.122452: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/sraj/tmpl7dq1vcg/assets
Saved pruned TFLite model to: /tmp/sraj/tmp_o3vyvsz.tflite


2023-10-13 00:17:12.485379: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-10-13 00:17:12.485431: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


Define a helper function to actually compress the models via gzip and measure the zipped size.



In [19]:
def get_gzipped_model_size(file):
    # Returns size of gzipped model, in bytes.
    import os
    import zipfile

    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

Compare and see that the models are 3x smaller from pruning.



In [20]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 78843.00 bytes
Size of gzipped pruned Keras model: 26593.00 bytes
Size of gzipped pruned TFlite model: 25528.00 bytes


## Create a 10x smaller model from combining pruning and quantization

You can apply post-training quantization to the pruned model for additional benefits.



In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/sraj/tmp51uksfp1/assets


INFO:tensorflow:Assets written to: /tmp/sraj/tmp51uksfp1/assets


Saved quantized and pruned TFLite model to: /tmp/sraj/tmp6v01czon.tflite
Size of gzipped baseline Keras model: 78843.00 bytes
Size of gzipped pruned and quantized TFlite model: 8796.00 bytes


2023-10-13 00:24:39.651777: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-10-13 00:24:39.652349: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


### See persistence of accuracy from TF to TFLite
Define a helper function to evaluate the TF Lite model on the test dataset.




In [27]:
import numpy as np

def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    
    # Run predictions on every y image in the "test" dataset.
    
    prediction_digits = []
    for i, test_image in enumerate(test_images):
        if i % 1000 == 0:
            print('Evaluated on {n} results so far.'.format(n=i))
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)
        
        # Run inference
        interpreter.invoke()
        
        # Post-Processing :Remove batch dimension and find the digit with highest probability
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)
        
    print('\n')
    
    # Compare prediction results with ground truth labels to calculate accuracy.
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

You evaluate the pruned and quantized model and see that the accuracy from TensorFlow persists to the TFLite backend.



In [28]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Pruned and quantized TFLite test_accuracy: 0.9518
Pruned TF test accuracy: 0.9516000151634216


## Export to ONNX

In [34]:
import tf2onnx


In [36]:
from tensorflow.keras.models import load_model

pruned_model = load_model('tmp4g0h8tr4.h5')

OSError: No file or directory found at tmp4g0h8tr4.h5

In [31]:
# Save to a file
with open('model.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())

AttributeError: 'tuple' object has no attribute 'SerializeToString'